In [84]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm


In [85]:
def get_moth(url):
    cricbuzz_highlights_url=url
    cricbuzz_highlights_url=cricbuzz_highlights_url.replace('cricket-full-commentary','cricket-scores')
    cricbuzz_highlights_url
    page=requests.get(cricbuzz_highlights_url)
    soup=BeautifulSoup(page.content)
    try: 
        win=soup.find('div',{'class':'cb-col cb-col-100 cb-min-stts cb-text-complete'}).text
        #cb-col cb-col-100 cb-mini-col cb-min-comp ng-scope
        #cb-link-undrln ng-binding
        a=soup.find('span',{'class':'cb-text-gray cb-font-12'})
        try:
            man_of_the_match=a.next_sibling.next_sibling.next_sibling.text
        except:
            man_of_the_match=None
    except:
        win = None
        man_of_the_match=None
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2022/{file_name}.csv')
    df['win']=win
    df['man_of_the_match']=man_of_the_match
    df.to_csv(f'files/2022/{file_name}.csv',index=False)
    


In [86]:
#get_moth('https://www.cricbuzz.com/cricket-full-commentary/69575/csk-vs-gt-final-indian-premier-league-2023')

In [89]:
links=[]
with open('match_links/2022_season_match_links.txt','r') as file:
    for line in file:
        links.append(line.strip())
for i in tqdm(links):
    get_moth(i)

100%|██████████| 74/74 [01:25<00:00,  1.15s/it]


In [88]:
links[44+29]

'https://www.cricbuzz.com/cricket-full-commentary/47605/gt-vs-rr-final-indian-premier-league-2022'